In [1]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

In [2]:
panorama_image = cv2.imread('image_1024_aligned_rgb.png')

In [4]:
# Функция преобразования, в которую передаются угол обзора(fov), угол вращения по горизонтали(theta),
# угол вращения по вертикали(phi), ширина(width), высота(height)

def GetPerspective(fov, theta, phi, width, height):
        
        # Высота и ширина панорамы
        equ_h = panorama_image.shape[0]
        equ_w = panorama_image.shape[1]
        
        # Координаты центра панорамы
        equ_cx = (equ_w - 1) / 2.0
        equ_cy = (equ_h - 1) / 2.0

        # Горизонтальное и вертикальное поля зрения
        wFOV = fov
        hFOV = float(height) / width * wFOV

        # Половины ширины и высоты поля зрения в радианах
        w_len = np.tan(np.radians(wFOV / 2.0))
        h_len = np.tan(np.radians(hFOV / 2.0))

        # Двумерные карты координат в пространстве 
        x_map = np.ones([height, width], np.float32)
        y_map = np.tile(np.linspace(-w_len, w_len,width), [height,1])
        z_map = -np.tile(np.linspace(-h_len, h_len,height), [width,1]).T

        # Расстояние от точек на сфере до начала координат
        D = np.sqrt(x_map**2 + y_map**2 + z_map**2)
        
        # Трехмерные координаты точек на сфере 
        xyz = np.stack((x_map,y_map,z_map),axis=2)/np.repeat(D[:, :, np.newaxis], 3, axis=2)
        
        # Векторы осей вращения по y и z
        y_axis = np.array([0.0, 1.0, 0.0], np.float32)
        z_axis = np.array([0.0, 0.0, 1.0], np.float32)
        
        # Матрица вращения для поворота вокруг y, z осей
        [R1, _] = cv2.Rodrigues(z_axis * np.radians(theta))
        [R2, _] = cv2.Rodrigues(np.dot(R1, y_axis) * np.radians(-phi))

        # Преобразование формы массива xyz для смены осей местами
        xyz = xyz.reshape([height * width, 3]).T
        
        # Вращение с помощью R1 и R2
        xyz = np.dot(R1, xyz)
        xyz = np.dot(R2, xyz).T
        
        # Широта и долгота точек на сфере
        lat = np.arcsin(xyz[:, 2])
        lon = np.arctan2(xyz[:, 1] , xyz[:, 0])

        # Преобразование в пиксельные координаты на плоскости панорамы 
        lon = lon.reshape([height, width]) / np.pi * 180
        lat = -lat.reshape([height, width]) / np.pi * 180

        # Преобразование широты и долготы с учетом центров equ_cx, equ_cy
        lon = lon / 180 * equ_cx + equ_cx
        lat = lat / 90  * equ_cy + equ_cy

        # Перспективное преобразование с использованием координат широты и долготы
        perspective = cv2.remap(panorama_image, lon.astype(np.float32), lat.astype(np.float32), cv2.INTER_CUBIC, borderMode=cv2.BORDER_WRAP)
        return perspective

In [ ]:
# Угол обзора 90 градусов для наименьших искажений
fov = 90 
width, height = 1650, 1650

wall1_image = GetPerspective(fov, 0, 0, width, height)
wall2_image = GetPerspective(fov, 90, 0, width, height)
wall3_image = GetPerspective(fov, 120, 0, width, height)
wall4_image = GetPerspective(fov, 160, 0, width, height)
wall5_image = GetPerspective(fov, 180, 0, width, height)
wall6_image = GetPerspective(fov, -90, 0, width, height)
floor_image = GetPerspective(fov, 0, -90, width, height)
ceil_image = GetPerspective(fov, 0, 90, width, height)

In [6]:
image_list = [wall1_image, wall2_image, wall3_image, wall4_image, wall5_image, wall6_image, floor_image, ceil_image]

In [ ]:
for image in image_list:
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
plt.show()

In [8]:
for i, image in enumerate(image_list, start=0):
        cv2.imwrite(f'{i}_proj.jpg', image)